# Preprocessing

In [1]:
import sys
import shutil
from pathlib import Path 

In [2]:
def preprocess_emoji(dset_path):
    # make emoji unicode vocabulary 
    code_vocaburary = {}
    code_path = Path('dataset/description/unicode.txt')
    code_list = code_path.read_text(encoding='utf-8').split('\n')
    for index, data in enumerate(code_list):
        code_vocaburary[data] = index
    
    # chack dataset path
    image_path = Path(dset_path)
    if image_path.exists() == False:
        exit('Check your dataset path!')

    # copy designated emoji images
    for filepath in list(image_path.glob("./**/64/**/*.png")):
        if str(filepath.name.split(".")[0]) in code_list:
            shutil.copyfile(filepath, \
                'dataset/edited/' + str(code_vocaburary[filepath.name.split(".")[0]]) + '.png')

In [3]:
DATASET_LOCATION = "dataset/original"
print("Started pre-processing")
preprocess_emoji(DATASET_LOCATION)
print("Completed pre-processing")

Started pre-processing
Completed pre-processing


# Model

In [4]:
# GPU setting
import tensorflow as tf
from tensorflow.keras.backend import set_session

config = tf.ConfigProto(
            gpu_options = tf.GPUOptions(
                visible_device_list="0", # specify GPU number
                allow_growth=True)
        )

print(config)

set_session(tf.Session(config=config))

gpu_options {
  allow_growth: true
  visible_device_list: "0"
}



In [5]:
from __future__ import print_function, division

from keras.layers import Input, Dense, Reshape, Flatten, Dropout, concatenate
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras import backend as kb
from keras.layers import Lambda
from utils.glove_loader import GloveModel
import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pyplot as plt
import numpy as np
from utils.dataset_utils import load_dataset
from PIL import Image
import math
import pandas as pd
import sys
import time

Using TensorFlow backend.


ModuleNotFoundError: No module named 'utils.glove_loader'

In [ ]:
class DCGAN():
    def __init__(self, img_path, txt_path, glove_path):
        # Input shape
        self.img_rows = 64
        self.img_cols = 64
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100
        self.embedding_dim = 300
        self.img_path = img_path
        self.txt_path = txt_path
        self.glove_path = glove_path

        optimizer_g = Adam(0.0005, 0.5)
        optimizer_d = Adam(0.00005, 0.5)

        # Build the GloVe model
        self.glove_model = GloveModel()
        self.glove_model.load(data_dir_path=self.glove_path, embedding_dim=self.embedding_dim)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer_d,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        cond_input = Input(shape=(self.embedding_dim,))
        img = self.generator([z, cond_input])

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator([img, cond_input])

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model([z, cond_input], valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer_g)

    def build_generator(self):
        generator_input = Input(shape=(self.latent_dim, ), name="g_input")
        cond_input = Input(shape=(self.embedding_dim, ), name="cond_g_input")
        cond_output = Dense(100)(cond_input)

        G = concatenate([generator_input, cond_output])
        G = Dense(256 * 8 * 8, activation="relu")(G)
        G = Reshape((8, 8, 256))(G)
        G = UpSampling2D()(G)
        G = Conv2D(256, kernel_size=3, padding="same")(G)
        G = BatchNormalization(momentum=0.8)(G)
        G = Activation("relu")(G)
        G = UpSampling2D()(G)
        G = Conv2D(128, kernel_size=3, padding="same")(G)
        G = BatchNormalization(momentum=0.8)(G)
        G = Activation("relu")(G)
        G = UpSampling2D()(G)
        G = Conv2D(64, kernel_size=3, padding="same")(G)
        G = BatchNormalization(momentum=0.8)(G)
        G = Activation("relu")(G)
        G = Conv2D(self.channels, kernel_size=3, padding="same")(G)
        generator_output = Activation("tanh")(G)

        generator = Model([generator_input, cond_input], generator_output)
        generator.summary()

        return generator

    def build_discriminator(self):
        discriminator_input = Input(shape=self.img_shape, name="d_input")
        cond_input = Input(shape=(self.embedding_dim, ), name="cond_d_input")
        D = Conv2D(64, kernel_size=3, strides=2, padding="same")(discriminator_input)
        D = LeakyReLU(alpha=0.2)(D)
        D = Dropout(0.25)(D)
        D = Conv2D(128, kernel_size=3, strides=2, padding="same")(D)
        D = ZeroPadding2D(padding=((0,1),(0,1)))(D)
        D = BatchNormalization(momentum=0.8)(D)
        D = LeakyReLU(alpha=0.2)(D)
        D = Dropout(0.25)(D)
        D = Conv2D(256, kernel_size=3, strides=1, padding="same")(D)
        D = BatchNormalization(momentum=0.8)(D)
        D = LeakyReLU(alpha=0.2)(D)
        D = Dropout(0.25)(D)
        D = Conv2D(512, kernel_size=3, strides=2, padding="same")(D)
        D = BatchNormalization(momentum=0.8)(D)
        D = LeakyReLU(alpha=0.2)(D)

        cond_d_hidden = Dense(100)(cond_input)
        cond_d_hidden = Reshape((1, 1, 100))(cond_d_hidden)
        cond_d_output = Lambda(lambda x: kb.tile(x, [1, 9, 9, 1]))(cond_d_hidden)

        D = concatenate([D, cond_d_output], axis=-1)
        D = Conv2D(512, kernel_size=3, strides=1, padding='same')(D)
        D = BatchNormalization(momentum=0.8)(D)
        D = LeakyReLU(alpha=0.1)(D)
        D = Dropout(0.25)(D)
        D = Flatten()(D)
        discriminator_output = Dense(1, activation='sigmoid')(D)

        discriminator = Model([discriminator_input, cond_input], discriminator_output)
        discriminator.summary()

        return discriminator